Imports:

In [2]:

from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch
from torch import nn
import torch.optim as optim
import torchvision
#from torchvision import datasets
from tensorboardX import SummaryWriter
#Visulization
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
#Others
#from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy
#from finetuneSAM.utils.dataset import Public_dataset
import torch.nn.functional as F
from torch.nn.functional import one_hot
from pathlib import Path
from tqdm import tqdm
#from finetuneSAM.utils.losses import DiceLoss
#from finetuneSAM.utils.dsc import dice_coeff_multi_class
import cv2
import monai
#from finetuneSAM.utils.utils import vis_image


# General
import numpy as np
import torch


from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch

import torchvision


import matplotlib.pyplot as plt

from PIL import Image


import matplotlib.pyplot as plt


from finetuneSAM.models.sam.utils.transforms import ResizeLongestSide
from finetuneSAM.models.sam import SamPredictor, sam_model_registry
from finetuneSAM.cfg import parse_args

from mmutils import put_marks, put_mask


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_5m_224 in registry with finetuneSAM.models.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.p

device(type='cuda')

Create Model:

In [14]:
#Fine-Tune Sam
args =  parse_args()


# setting if_mask_decoder_adapter = True puts adapters inside 2-way transformer blocks
# this does not change the number of decoder 2-way transformer blocks (def = 2)
# decoder_adapt_depth denotes how many of the two 2-way transformer blocks are adapted


# setting if_encoder_adapter = True puts adapters inside TinyViTBlocks in the encoder
# this does not change the number of encoder TinyViTBlocks (def = 4)
# encoder_adapt_depth (e.g. [1,2]) denotes how deep blocks will be adapted

args.finetune_type = "vanilla"
#args.finetune_type = "adapter"# "vanilla"
#args.if_mask_decoder_adapter = True
#args.image_size = 512
#args.decoder_adapt_depth = 1
args.num_cls = 2


# Load the pre-trained model
sam_checkpoint = "mobile_sam.pt"
# Define the model type: Tiny Vit
model_type = "vit_t"

# Load the model
mobile_sam_f = sam_model_registry[model_type](args , checkpoint=sam_checkpoint)
# Move the model to the device
mobile_sam_f = mobile_sam_f.to(device=device)
# Set the model to evaluation mode
#mobile_sam_f.eval()

predictor_f = SamPredictor(mobile_sam_f)

c:\KU\ATIA\finetuneSAM\models\sam\build_sam.py:158: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Predcit def:

In [ ]:
def predict(model, input_img_size, image: np.ndarray, point = None):
    torch.no_grad()
    org_shape = image.shape

    transform = ResizeLongestSide(input_img_size) # can be changed?

    #set_image
    input_image = transform.apply_image(image)
    input_image_torch = torch.as_tensor(input_image, device=device)
    input_image_torch = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]

    #set_torch_image
    transformed_image = input_image_torch
    input_size = tuple(transformed_image.shape[-2:])
    transformed_image = model.preprocess(transformed_image)
    features = model.image_encoder(transformed_image)


    coords_torch, labels_torch, box_torch, mask_input_torch = None, None, None, None

    # TO DELETE
    t1 = np.array([70,235])
    t2 = np.array([218,92])
    t3 = np.array([154,360])
    ts = np.array([t1,t2,t3])
    # # background
    b1 = np.array([259,257])
    b2 = np.array([192,148])
    b3 = np.array([220,435])
    bs = np.array([b1,b2,b3])
    ps = np.array([t1,t2,t3,b1,b2,b3])

    labels = np.array([1,1,1,0,0,0])
    


    point_coords = transform.apply_coords(ps, org_shape[:2])
    coords_torch = torch.as_tensor(point_coords, dtype=torch.float, device=device)
    labels_torch = torch.as_tensor(labels, dtype=torch.int, device=device)
    coords_torch, labels_torch = coords_torch[None, :, :], labels_torch[None, :]


    points=(coords_torch, labels_torch)
    sparse_embeddings, dense_embeddings = model.prompt_encoder(
            points=points,
            boxes=box_torch,
            masks=mask_input_torch,
        )

    # Predict masks
    low_res_masks, iou_predictions = model.mask_decoder(
        image_embeddings = features,
        image_pe= model.prompt_encoder.get_dense_pe(),
        sparse_prompt_embeddings=sparse_embeddings,
        dense_prompt_embeddings=dense_embeddings,
        multimask_output= True,
    )

    # Upscale the masks to the original image resolution
    masks = model.postprocess_masks(low_res_masks, input_size, org_shape[:2])


    return masks, iou_predictions, low_res_masks
    #return masks[0].detach().cpu().numpy(), iou_predictions[0].detach().cpu().numpy(), low_res_masks[0].detach().cpu().numpy()

In [45]:
image = Image.open('data/Data/test/image/0.png')
#image = Image.open('eye_enchanced.png')
print(image.size)
image = np.asarray(image)
#image = image[0:254,0:254]
#image = image[0:1024,0:1024]
print(image.shape)



image_tru = Image.open('data/Data/test/mask/0.png')
print(image_tru.size)
image_tru = np.asarray(image_tru)
image_tru.shape


(512, 512)
(512, 512, 3)
(512, 512)


(512, 512)

In [46]:
masks, scores, low_masks = predict(mobile_sam_f, 512, image)
print(masks.shape)
print(scores)
#results = format_results(masks, scores, logits, filter=0)
#results[0]

torch.Size([1, 2, 512, 512])
tensor([[0.0805, 0.1544]], device='cuda:0', grad_fn=<SliceBackward0>)


In [9]:
masks = masks.detach().cpu()
print(type(masks))  
print(masks.shape)
image_tru = np.where(image_tru>0,1,0)
image_tru = torch.tensor([image_tru]).long()
print(type(image_tru))
print(image_tru.shape)




<class 'torch.Tensor'>
torch.Size([1, 2, 512, 512])
<class 'torch.Tensor'>
torch.Size([1, 512, 512])


C:\Users\newma\AppData\Local\Temp\ipykernel_23168\2027797337.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  image_tru = torch.tensor([image_tru]).long()


In [63]:
ones = len(np.where(image_tru>0)[0])
zeros = len(np.where(image_tru==0)[0])

print(image_tru.shape)  
print(np.unique(image_tru))

l0 = np.where(image_tru[0] < 0, 1, 0)
l1 = np.where(image_tru[0] > 0, 1, 0)
dummy_mask = np.zeros((1,2,512,512))
dummy_mask[0][0] = l0
dummy_mask[0][1] = l1
dummy_mask = torch.tensor(dummy_mask).float()
print(dummy_mask.shape)




torch.Size([1, 512, 512])
[0 1]
torch.Size([1, 2, 512, 512])


In [42]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.126,1]).float())
loss = criterion(dummy_mask, image_tru)
loss

tensor(0.4905)

In [12]:
predictor_f.set_image(image)
FEAT_AUTO = predictor_f.features
#masks, scores, logits = predictor_f.predict(None,None)#, multimask_output=True)

# 70,235  218, 92     154,360 
# 259,257    192, 148  220,435

# doggo
# p1 = np.array([940,730]) 
# p2 = np.array([1500,850]) 
# p3 = np.array([1100,400])
# ps = np.array([p1,p2,p3])

#0.png
# target
t1 = np.array([70,235])
t2 = np.array([218,92])
t3 = np.array([154,360])
ts = np.array([t1,t2,t3])
# background
b1 = np.array([259,257])
b2 = np.array([192,148])
b3 = np.array([220,435])
bs = np.array([b1,b2,b3])
ps = np.array([t1,t2,t3,b1,b2,b3])

labels = np.array([1,1,1,0,0,0])

masks, scores, logits = predictor_f.predict(ps,point_labels= labels)
# p,point_labels= np.array([1])
scores

input identical True
input_image_torch(after cast):  True
input_image_torch (after permute)  True
input_image after preproc:  True
features identical True


array([0.7350938 , 0.91377735, 0.97557175], dtype=float32)

In [34]:


q = put_mask(image,masks[0])

q = put_marks(q, ps[3:], 10)
q = put_marks(q, ps[0:3], 10, target=True)



Image.fromarray(q).show()

In [ ]:
from os import listdir
from os.path import isfile, join
from os import getcwd
from generate_target_ps import choose_bg_points, choose_target_points

def load_data(path):
    curr_dir = getcwd()
    # images
    imgs_path = curr_dir+'/data/Data/train/image/'
    imgs_names = [f for f in listdir(imgs_path) if isfile(join(imgs_path, f))]
    # masks
    masks_path = curr_dir+'/data/Data/train/mask/'
    msks_names = [f for f in listdir(masks_path) if isfile(join(masks_path, f))]

    data = {'image': [], 'mask': [], 't_points': [], 'bg_points': []}



    for img_name in imgs_names:
        img = Image.open(imgs_path+img_name)
        img = np.asarray(img)
        data['image'].append(img)
    
    ts = 5
    bgs = 5

    for msk_name in msks_names:
        msk = Image.open(masks_path+msk_name)
        msk = np.asarray(msk)
        t_points = choose_target_points(msk, ts, min_dist=50)
        bg_points = choose_bg_points(msk, bgs, min_dist=50)
        data['t_points'].append(t_points)
        data['bg_points'].append(bg_points)
        data['mask'].append(msk)
    return data
        
        


In [33]:
data = load_data('data/Data/train/')

distance applied:  45
distance applied:  45
distance applied:  45
distance applied:  45


In [34]:
data['image'][0].shape

torch.Size([1, 512, 512, 3])

In [43]:


sam = mobile_sam_f
train_points = len(data['image'])
checkpoints_path = 'checkpoints'


if args.finetune_type == 'adapter':
    for n, value in sam.named_parameters():
        if "Adapter" not in n: # only update parameters in adapter
            value.requires_grad = False
    print('if update encoder:',args.if_update_encoder)
    print('if image encoder adapter:',args.if_encoder_adapter)
    print('if mask decoder adapter:',args.if_mask_decoder_adapter)
    if args.if_encoder_adapter:
        print('added adapter layers:',args.encoder_adapter_depths)
    
# elif args.finetune_type == 'vanilla' and args.if_update_encoder==False:   
#     print('if update encoder:',args.if_update_encoder)
#     for n, value in sam.image_encoder.named_parameters():
#         value.requires_grad = False
# elif args.finetune_type == 'lora':
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder lora:',args.if_encoder_lora_layer)
#     print('if mask decoder lora:',args.if_decoder_lora_layer)
#     sam = LoRA_Sam(args,sam,r=4).sam
# sam.to('cuda')
b_lr = args.lr
epochs = 80
    
optimizer = optim.AdamW(sam.parameters(), lr=b_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay
criterion1 = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, to_onehot_y=True,reduction='mean')
criterion2 = nn.CrossEntropyLoss()
iter_num = 0
max_iterations = epochs * train_points
writer = SummaryWriter(checkpoints_path + '/log')

pbar = tqdm(range(epochs))
val_largest_dsc = 0
last_update_epoch = 0
for epoch in pbar:
    sam.train()
    train_loss = 0
    # batches here?
    #for i,_ in enumerate(tqdm(data['image'])):
    for i in range(1):
        imgs = data['image'][i][0].cuda()
        imgs = imgs.permute(2, 0, 1).contiguous()[None, :, :, :]
        imgs = imgs.float()
        msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][i])
        msks = msks.cuda()



        if args.if_update_encoder:
            img_emb = sam.image_encoder(imgs)
        else:
            with torch.no_grad():
                img_emb = sam.image_encoder(imgs)
        
        # get default embeddings
        sparse_emb, dense_emb = sam.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        pred, _ = sam.mask_decoder(
                        image_embeddings=img_emb,
                        image_pe=sam.prompt_encoder.get_dense_pe(), 
                        sparse_prompt_embeddings=sparse_emb,
                        dense_prompt_embeddings=dense_emb, 
                        #multimask_output=True,
                        multimask_output=False,
                        )

        #loss_dice = criterion1(pred,msks.float()) 
        loss_ce = criterion2(pred,torch.squeeze(msks.long(),1))
        loss =  loss_ce
        #loss =  loss_dice + loss_ce
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if args.if_warmup and iter_num < args.warmup_period:
            lr_ = args.lr * ((iter_num + 1) / args.warmup_period)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_

        else:
            if args.if_warmup:
                shift_iter = iter_num - args.warmup_period
                assert shift_iter >= 0, f'Shift iter is {shift_iter}, smaller than zero'
                lr_ = args.lr * (1.0 - shift_iter / max_iterations) ** 0.9  # learning rate adjustment depends on the max iterations
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_
            else:
                lr_ = args.lr

        train_loss += loss.item()
        iter_num+=1
        writer.add_scalar('info/lr', lr_, iter_num)
        writer.add_scalar('info/total_loss', loss, iter_num)
        writer.add_scalar('info/loss_ce', loss_ce, iter_num)
        #writer.add_scalar('info/loss_dice', loss_dice, iter_num)

    train_loss /= (i+1)
    pbar.set_description('Epoch num {}| train loss {} \n'.format(epoch,train_loss))

    # if epoch%2==0:
    #     eval_loss=0
    #     dsc = 0
    #     sam.eval()
    #     with torch.no_grad():
    #         for i in tqdm(data['image']):
    #             imgs = data['image'][i].cuda()
    #             msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][i])
    #             msks = msks.cuda()

    #             img_emb= sam.image_encoder(imgs)
    #             sparse_emb, dense_emb = sam.prompt_encoder(
    #                 points=None,
    #                 boxes=None,
    #                 masks=None,
    #             )
    #             pred, _ = sam.mask_decoder(
    #                             image_embeddings=img_emb,
    #                             image_pe=sam.prompt_encoder.get_dense_pe(), 
    #                             sparse_prompt_embeddings=sparse_emb,
    #                             dense_prompt_embeddings=dense_emb, 
    #                             multimask_output=True,
    #                             )
    #             loss = criterion1(pred,msks.float()) + criterion2(pred,torch.squeeze(msks.long(),1))
    #             eval_loss +=loss.item()
    #             dsc_batch = dice_coeff_multi_class(pred.argmax(dim=1).cpu(), torch.squeeze(msks.long(),1).cpu().long(),args.num_cls)
    #             dsc+=dsc_batch

    #         eval_loss /= (i+1)
    #         dsc /= (i+1)
            
    #         writer.add_scalar('eval/loss', eval_loss, epoch)
    #         writer.add_scalar('eval/dice', dsc, epoch)
            
    #         print('Eval Epoch num {} | val loss {} | dsc {} \n'.format(epoch,eval_loss,dsc))
    #         if dsc>val_largest_dsc:
    #             val_largest_dsc = dsc
    #             last_update_epoch = epoch
    #             print('largest DSC now: {}'.format(dsc))
    #             torch.save(sam.state_dict(),checkpoints_path + '/checkpoint_best.pth')
    #         elif (epoch-last_update_epoch)>20:
    #             # the network haven't been updated for 20 epochs
    #             print('Training finished###########')
    #             break
writer.close()

  0%|          | 0/80 [00:00<?, ?it/s]


AssertionError: input feature has wrong size